In [34]:
import tensorflow as tf
import tensorflow.keras.datasets as tfds
from utils.layer_units import *
import pydotplus
import numpy as np

# Plot configurations
%matplotlib inline

# Notebook auto reloads code. (Ref: http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
(X_train,y_train),(X_test,y_test) = tfds.cifar100.load_data()

In [36]:
X_valid = X_train[49000:]
y_valid = y_train[49000:]
X_train = X_train[:49000]
y_train = y_train[:49000]

In [37]:
# center but not scale data
# we found scale not very stable
def preprocess(X):
    # pixel wise center
    ret = X.astype(np.float32) - np.mean(X,axis=0)
    return ret

In [38]:
X_train = preprocess(X_train)
X_valid = preprocess(X_valid)
X_test = preprocess(X_test)

In [39]:
ipt = tf.keras.Input(shape=(32,32,3))
resid_params = [[128,3,1,0],[128,3,1,0]]
#ipt = tf.keras.Conv2D(filter)
attention_module1 = attention_unit(ipt,resid_layer_params = resid_params,skip_connection=True, skip_mode = 1)
resid1 = residual_units(attention_module1,resid_params)
attention_module2 = attention_unit(resid1,resid_layer_params = resid_params,skip_connection=True, skip_mode = 1)
resid2 = residual_units(attention_module2,resid_params)
attention_module3 = attention_unit(resid2,resid_layer_params = resid_params,skip_connection=True, skip_mode = 1)
resid3 = residual_units(attention_module3,resid_params)
resid4 = residual_units(resid3,resid_params)
avg0 = tf.keras.layers.GlobalAveragePooling2D()(resid4)
#flat1 = tf.keras.layers.Flatten()(avg0)
#dense0 = tf.keras.layers.Dense(256,activation='relu')(avg0)
#dense1 = tf.keras.layers.Dense(256,activation='relu')(avg0)
dense2 = tf.keras.layers.Dense(100,activation='softmax')(avg0)

model = tf.keras.Model(ipt,outputs=dense2)
opt = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9,nesterov=True)

model.compile(optimizer = 'Adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['sparse_categorical_accuracy'],
)
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
batch_normalization_310 (BatchN (None, 32, 32, 3)    12          input_6[0][0]                    
__________________________________________________________________________________________________
activation_325 (Activation)     (None, 32, 32, 3)    0           batch_normalization_310[0][0]    
__________________________________________________________________________________________________
conv2d_495 (Conv2D)             (None, 32, 32, 128)  3584        activation_325[0][0]             
____________________________________________________________________________________________

In [40]:
# ImageDataGenerator code referred from hw2 Task4
# and referred from https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
datagen = tf.keras.preprocessing.image.ImageDataGenerator(shear_range=0.1,zoom_range=0.2,horizontal_flip=True,rotation_range=30,width_shift_range=0.1,
                             height_shift_range=0.1)
datagen.fit(X_train)

In [41]:
#history = model.fit(X_train,y_train,epochs=10,batch_size=128,validation_split=0.2,callbacks=[tensorboard_callback])
bs = 128
history = model.fit(datagen.flow(X_train,y_train,batch_size=bs),epochs=10,steps_per_epoch=len(X_train)//bs,
                    validation_data = (X_valid,y_valid))

Epoch 1/10
382/382 [==============================] - 435s 1s/step - loss: 7.4541 - sparse_categorical_accuracy: 0.0620 - val_loss: 4.0747 - val_sparse_categorical_accuracy: 0.0770
Epoch 2/10
382/382 [==============================] - 435s 1s/step - loss: 3.8147 - sparse_categorical_accuracy: 0.1190 - val_loss: 3.9798 - val_sparse_categorical_accuracy: 0.1230
Epoch 3/10
382/382 [==============================] - 434s 1s/step - loss: 3.5446 - sparse_categorical_accuracy: 0.1625 - val_loss: 3.7918 - val_sparse_categorical_accuracy: 0.1490
Epoch 4/10
382/382 [==============================] - 434s 1s/step - loss: 3.3047 - sparse_categorical_accuracy: 0.1975 - val_loss: 4.4305 - val_sparse_categorical_accuracy: 0.1160
Epoch 5/10
382/382 [==============================] - 434s 1s/step - loss: 3.0898 - sparse_categorical_accuracy: 0.2355 - val_loss: 3.9176 - val_sparse_categorical_accuracy: 0.1960
Epoch 6/10
382/382 [==============================] - 435s 1s/step - loss: 2.8884 - sparse_cate

In [43]:
import pandas as pd
hist_normalized = pd.DataFrame(history.history)

In [44]:
hist_normalized.to_csv('hist_double_128_3_3_no_dense.csv',sep=',')